In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
from time import time
import re
import string
import os
import emoji
from pprint import pprint
import collections
import glob

from sklearn.base import BaseEstimator, TransformerMixin

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english')) 

In [2]:
def adapt_valence_scores(df, column_name): 
    valence_list = list()
    for index, row in df.iterrows():
        valence = row[column_name]
        if column_name == 'Intensity Class':   # If oc then need to alter valence representation
            valence = valence.replace(valence, valence[:2].replace(":", ""))
        valence_list.append(valence)
    return valence_list


In [3]:
class CleanText(BaseEstimator, TransformerMixin):
    """
    From https://towardsdatascience.com/sentiment-analysis-with-text-mining-13dd2b33de27
    """
    def remove_mentions(self, input_text):
        return re.sub(r'@\w+', '', input_text)
    
    def remove_urls(self, input_text):
        return re.sub(r'http.?://[^\s]+[\s]?', '', input_text)
    
    def emoji_oneword(self, input_text):
        # By compressing the underscore, the emoji is kept as one word
        return input_text.replace('_','')
    
    def remove_punctuation(self, input_text):
        # Make translation table
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
        return input_text.translate(trantab)
    
    def remove_digits(self, input_text):
        return re.sub('\d+', '', input_text)
    
    def to_lower(self, input_text):
        return input_text.lower()
    
    def remove_stopwords(self, input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        clean_X = X.apply(self.remove_mentions).apply(self.remove_urls).apply(self.emoji_oneword).apply(self.remove_punctuation).apply(self.remove_digits).apply(self.to_lower).apply(self.remove_stopwords)
        return clean_X

In [5]:
folder_path = 'data/raw/'
out_folder_path = 'data/cleaned/'

#if __name__ == '__main__':
raw_filepaths = glob.glob(folder_path + '*')
# Loop over raw files
for filepath in raw_filepaths:
    basename = os.path.basename(filepath)
    basename_ = basename.split('-') # Get list of splitted basename for use later

    # Read in df
    df = pd.read_table(filepath)
    
    # Get valence score
    if basename_[2] == 'oc':
        val_col = "Intensity Class"
    elif basename_[2] == 'reg':
        val_col = "Intensity Score"
    valence_list = adapt_valence_scores(df, val_col)
    
    
    # If data is English
    if basename_[3] == 'En':
        ct = CleanText()
        clean_ = ct.fit_transform(df.Tweet)
    

    # If data is Arabic
    elif basename_[3] == 'Ar':
        continue
        
    # Fill empty cells with '[no_text]'
    empty_clean = clean_ == ''
    if clean_[empty_clean].count() > 0:
        print(f'{clean_[empty_clean].count()} records have no words left after text cleaning in {basename}')
        clean_.loc[empty_clean] = '[no_text]'
        
    # Create new df of cleaned data
    df_cleaned = pd.DataFrame(clean_)
    # Add valence score to list
    df_cleaned['Valence score'] = valence_list
        
    # Drop rows with no tweet text
    df_cleaned = df_cleaned[df_cleaned['Tweet'] != '[no_text]']
    df_cleaned.reset_index(drop=True, inplace=True)
        
    # Write to csv
    outpath = out_folder_path + '_'.join(['cleaned', basename_[1], basename_[2], basename_[3], basename_[-1]])
    # For example "data/cleaned/cleaned_Valence_oc_En_train.txt'
    df_cleaned.to_csv(outpath, sep="\t")

#df_cleaned

arabic


C:\Users\quiri\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  if sys.path[0] == '':


ValueError: Length of values does not match length of index